In [1]:
#tabular data imports :
import pandas as pd
import numpy as np
from pydataset import data

# visualization imports:
import matplotlib.pyplot as plt
import seaborn as sns
# Custom palette with specified colors
custom_palette = sns.color_palette(["#33B8C6", "#C64133", "#4682B4"])
# Using a matplotlib colormap
custom_palette2 = sns.color_palette(["#33B8C6","#50C2CE","#6DCCD6","#8AD6DE","#A7E0E6","#C4EAEE","#E1F4F6"])

from sklearn.cluster import KMeans

from sklearn.model_selection import train_test_split
import sklearn.preprocessing
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.impute import SimpleImputer

# success metrics from earlier in the week: mean squared error and r^2 explained variance
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures

#stats
from scipy import stats
from scipy.stats import pearsonr, spearmanr
from scipy.stats import shapiro

import warnings
warnings.filterwarnings("ignore")
import wrangle as w
import explore as e
import model as m
import os
directory = os.getcwd()
pd.set_option('display.max_columns', None)

In [2]:
# Calling the function that combines all the steps before and creates a .csv file from the df (to run faster and locally)
df = w.prepare_wine(w.red_wine_url, w.white_wine_url)
df.head(3)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,color,wine_quality
wine_id,,,,,,,,,,,,,,
w-1,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,white,average
w-2,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,white,average
w-3,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,white,average


In [3]:
train_prepped, val_prepped, test_prepped = m.processed_wine(df)

In [4]:
train_prepped.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,white
wine_id,,,,,,,,,,,,,
w-4568,0.165289,0.133333,0.397590,0.130368,0.049834,0.171821,0.424411,0.181029,0.728682,0.185393,0.405797,0.333333,1.0
w-3787,0.297521,0.140000,0.289157,0.187117,0.046512,0.415808,0.488211,0.211490,0.232558,0.067416,0.115942,0.333333,1.0
r-834,0.644628,0.260000,0.265060,0.015337,0.229236,0.240550,0.124827,0.216888,0.511628,0.359551,0.275362,0.166667,0.0
r-550,0.429752,0.300000,0.295181,0.019939,0.269103,0.034364,0.052705,0.200308,0.426357,0.219101,0.202899,0.500000,0.0
w-4828,0.239669,0.093333,0.198795,0.009202,0.044850,0.240550,0.221914,0.066898,0.294574,0.303371,0.492754,0.500000,1.0


In [5]:
train_scaled, validate_scaled, test_scaled = e.preprocess_wine(df)


    train -> (3898, 16)
    validate -> (1299, 16)
    test -> (1300, 16)


In [6]:
train_scaled.head(3)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,white,average,high,low
wine_id,,,,,,,,,,,,,,,,
w-2549,0.165217,0.144,0.42,0.099693,0.055092,0.419244,0.563107,0.160594,0.362205,0.169492,0.217391,0.500000,1.0,1.0,0.0,0.0
w-4295,0.260870,0.112,0.28,0.128834,0.056761,0.144330,0.319001,0.145942,0.283465,0.180791,0.405797,0.333333,1.0,1.0,0.0,0.0
r-956,0.356522,0.104,0.52,0.019939,0.130217,0.054983,0.047157,0.180644,0.488189,0.248588,0.347826,0.333333,0.0,1.0,0.0,0.0


In [7]:
train_scaled.quality.value_counts()

0.500000    1701
0.333333    1283
0.666667     647
0.166667     130
0.833333     116
0.000000      18
1.000000       3
Name: quality, dtype: int64

In [8]:
X_train, y_train = train_prepped.drop(columns=['quality']), train_prepped['quality']
X_validate, y_validate = val_prepped.drop(columns=['quality']), val_prepped['quality']
X_test, y_test = test_prepped.drop(columns=['quality']), test_prepped['quality']

# Modeling

In [9]:
# setting values for train group
X = X_train
y = y_train
# BASELINE
baseline = y.mode()
round(baseline,2)

0    0.5
Name: quality, dtype: float64

In [10]:
# making a baseline guess dataframe
# that is the same shape as our y variable:
# np.repeat does what it sounds like
baseline_array = np.repeat(baseline, y.shape[0])
baseline_rmse, baseline_r2 = m.evaluate_reg(y, baseline_array)

In [11]:
# setting up a dictionary to add the results of the models
eval_df = pd.DataFrame([{
    'Model': 'baseline',
    'rmse': baseline_rmse,
    'r2': baseline_r2
}])
eval_df

,Model,rmse,r2
0,baseline,0.150263,-0.046374


## MODEL: LassoLars

- train
- Validate